# CrewAI with AgentNeo Integration

This Jupyter notebook demonstrates the integration of AgentNeo, a powerful tracing and monitoring tool, with CrewAI, a framework for orchestrating role-playing AI agents. This integration allows for comprehensive analysis and debugging of AI-powered systems.

## Setup and Imports

First, let's import the necessary libraries and set up our environment.

In [1]:
import os
os.chdir('../..')

In [2]:
from crewai import Agent, Task, Crew, Process
from dotenv import load_dotenv

from agentneo import AgentNeo, Tracer

# Load environment variables
load_dotenv("YOUR_ENV_FILE_PATH")

False

## Initialize AgentNeo Session and Tracer

Now, let's set up our AgentNeo session and tracer.

In [3]:
# Create a session
neo_session = AgentNeo(session_name="crewai_example")

# Connect to a Project
neo_session.create_project(project_name="crewai_example_project")

# Create tracer
tracer = Tracer(session=neo_session)

tracer.start()

ValueError: Project 'crewai_example_project' already exists.

## Define Helper Functions

Let's define some helper functions for our AI tools, using AgentNeo's tracing capabilities.

In [4]:
from langchain_community.tools import TavilySearchResults

travily_tool = tracer.wrap_langchain_tool(TavilySearchResults())

## Define Agents

Now, let's create our AI agents using CrewAI.

In [5]:
@tracer.trace_agent("researcher")
def create_researcher():
    return Agent(
        role='Senior Research Analyst',
        goal='Uncover cutting-edge developments in AI and data science',
        backstory="""You work at a leading tech think tank.
        Your expertise lies in identifying emerging trends.
        You have a knack for dissecting complex data and presenting actionable insights.""",
        verbose=True,
        allow_delegation=False,
        tools=[travily_tool],
    )

@tracer.trace_agent("writer")
def create_writer():
    return Agent(
        role='Tech Content Strategist',
        goal='Craft compelling content on tech advancements',
        backstory="""You are a renowned Content Strategist, known for your insightful and engaging articles.
        You transform complex concepts into compelling narratives.""",
        verbose=True,
        allow_delegation=True,
    )

researcher = create_researcher()
writer = create_writer()

DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 18
DEBUG:agentneo.tracing.agent_tracer:Successfully updated and committed AgentCallModel with id 19


## Define Tasks

Let's create tasks for our agents.

In [6]:
task1 = Task(
  description="""Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.""",
  expected_output="Full analysis report in bullet points",
  agent=researcher
)

task2 = Task(
  description="""Using the insights provided, develop an engaging blog
  post that highlights the most significant AI advancements.
  Your post should be informative yet accessible, catering to a tech-savvy audience.
  Make it sound cool, avoid complex words so it doesn't sound like AI.""",
  expected_output="Full blog post of at least 4 paragraphs",
  agent=writer
)

## Create and Execute Crew

Now, let's create our crew and execute the tasks.

In [7]:
crew = Crew(
  agents=[researcher, writer],
  tasks=[task1, task2],
  process=Process.sequential,
  verbose=True,
)

# Get your crew to work!
result = crew.kickoff()

print(result)

tracer.stop()

13:12:26 - LiteLLM:INFO: utils.py:2740 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


# Agent: Senior Research Analyst
## Task: Conduct a comprehensive analysis of the latest advancements in AI in 2024.
  Identify key trends, breakthrough technologies, and potential industry impacts.


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
13:12:27 - LiteLLM:INFO: utils.py:938 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
13:12:27 - LiteLLM:INFO: utils.py:2740 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


 

I encountered an error while trying to use the tool. This was the error: Too many arguments to single-input tool tavily_search_results_json.
                Consider using StructuredTool instead. Args: ['Latest Advancements in AI 2024', 'Search for the most recent developments in artificial intelligence, including trends, technologies, and industry impacts.'].
 Tool tavily_search_results_json accepts these inputs: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.



# Agent: Senior Research Analyst
## Thought: I need to gather information on the latest advancements in AI for 2024. I will use the available tool to conduct a search on this topic.
## Using tool: tavily_search_results_json
## Tool Input: 
"{\"name\": \"Latest Advancements in AI 2024\", \"description\": \"Search for the most recent developments in artificial intelligence, including trends, technolo

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
13:12:29 - LiteLLM:INFO: utils.py:938 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
13:12:29 - LiteLLM:INFO: utils.py:2740 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


 

I encountered an error while trying to use the tool. This was the error: Too many arguments to single-input tool tavily_search_results_json.
                Consider using StructuredTool instead. Args: ['Latest advancements in AI 2024', 'Identifying key trends, breakthrough technologies, and potential industry impacts in the field of artificial intelligence for the year 2024.'].
 Tool tavily_search_results_json accepts these inputs: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.



# Agent: Senior Research Analyst
## Thought: Thought: I need to conduct a search to find out the latest advancements in AI for 2024, focusing on key trends, breakthrough technologies, and potential industry impacts.
## Using tool: tavily_search_results_json
## Tool Input: 
"{\"name\": \"Latest advancements in AI 2024\", \"description\": \"Identifying key trends, breakthrough tech

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
13:12:31 - LiteLLM:INFO: utils.py:938 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
13:12:31 - LiteLLM:INFO: utils.py:2740 - 
LiteLLM completion() model= gpt-4o-mini; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o-mini; provider = openai


 

I encountered an error while trying to use the tool. This was the error: Too many arguments to single-input tool tavily_search_results_json.
                Consider using StructuredTool instead. Args: ['advancements in AI 2024', 'Discover key trends, technologies, and industry impacts in artificial intelligence for 2024.'].
 Tool tavily_search_results_json accepts these inputs: A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.



# Agent: Senior Research Analyst
## Thought: Thought: I need to gather current information on the latest advancements in AI for 2024. I will initiate a search to identify current trends, breakthrough technologies, and their potential impacts on various industries.
## Using tool: tavily_search_results_json
## Tool Input: 
"{\"name\": \"advancements in AI 2024\", \"description\": \"Discover key trends, technologies, and industry impact

KeyboardInterrupt: 

## Stop Tracing and Launch Dashboard

Finally, let's stop the tracer and launch the AgentNeo dashboard to analyze the results.

In [ ]:
neo_session.launch_dashboard()

This notebook demonstrates how to integrate AgentNeo's tracing capabilities with CrewAI, allowing for detailed analysis and monitoring of AI agent interactions and task execution. The AgentNeo dashboard provides valuable insights into the performance and behavior of your AI system.